# Paso 1: Obtencion de datos

In [1]:
import GetOldTweets3 as got


def scrapper(initdate, finaldate ,toptweets, maximo, words=[]):
    s1=[]
    lista_final=[]
    for word in words:
        try:
            s1.append(got.manager.TweetCriteria().setQuerySearch(word).setSince(initdate).setUntil(finaldate).setTopTweets(toptweets).setMaxTweets(int(maximo)))
        except: continue
    
    for i in range (len(s1)):
        lista_final.append(got.manager.TweetManager.getTweets(s1[i]))
    return lista_final

In [2]:
#KeyWords for each candidate
res0= ['Espert', '#Despertar', '@jlespert', '@luisrosalesARG','@FrenteDespertar', '#DejenCompetirAEspert']
res1=['macri', 'pichetto', '#JUNTOSPORELCAMBIO', '@MAURICIOMACRI', '@MIGUELPICHETTO', '@JUNTOSCAMBIOAR']
res2=['@cfkargentina', '@alferdez', 'fernandez-fernandez', '@frentedetodos', 'alberto fernandez']
res3=['@RLavagna', '#consensofederal', 'urtubey']
res4=['Nicolas del caño', '#FITunidad', '@FTE_izquierda', '@NicolasdelCano', '#YoVoteFITUnidad', '@RominaDelPla']
res5=['@juanjomalvinas', '#FrenteNOS']

In [3]:
resultado0=scrapper("2019-08-10", "2019-08-11",True, 3500, res0)#Espert
resultado1=scrapper("2019-08-10", "2019-08-11",True, 3500, res1)#Macri
resultado2=scrapper("2019-08-10", "2019-08-11",True, 3500, res2)#CFK
resultado3=scrapper("2019-08-10", "2019-08-11",True, 3500, res3)#Lavagna
resultado4=scrapper("2019-08-10", "2019-08-11",True, 3500, res4)#Delcaño
resultado5=scrapper("2019-08-10", "2019-08-11",True, 3500, res5)#malvinas

In [4]:
def extractor(tweets):
    l1,l2,l3,l4,l5,l6=[],[],[],[],[],[]

    for i in range(len(tweets)):
        for tweet in tweets[i]:
            l1.append(tweet.text)
            l2.append(tweet.date)
            l3.append(tweet.username)
            l4.append(tweet.hashtags)
            l5.append(tweet.favorites)
            l6.append(tweet.retweets)
        
    final=pd.DataFrame(list(zip(l1, l2,l3,l4,l5,l6)), 
                       columns =['Tweets', 'Date','User', 'hashtags', 'Favs','RT'])

    
    return final

In [5]:
import pandas as pd

df0=extractor(resultado0)
df1=extractor(resultado1)
df2=extractor(resultado2)
df3=extractor(resultado3)
df4=extractor(resultado4)
df5=extractor(resultado5)

In [6]:
df0.head()

,Tweets,Date,User,hashtags,Favs,RT
0,La solución es #EspertRosales2019 #YoVotoEsper...,2019-08-10 23:59:58+00:00,Ivan78229914,#EspertRosales2019 #YoVotoEspert #ChauViejaPol...,21,2
1,espert maybe,2019-08-10 23:25:06+00:00,luisibiagetti,,0,0
2,Final #argentina #PASO2019 predictions: @mauri...,2019-08-10 23:13:12+00:00,speevr,#argentina #PASO2019 #Espert #ArgentinaVota,28,7
3,¡Desde España le deseamos mucha suerte al @Fre...,2019-08-10 22:46:24+00:00,libtv_,#YoVotoEspertpic,75,20
4,#EspertRosales2019 @jlespert Presidente !! @lu...,2019-08-10 22:36:02+00:00,EzequielDQuiro1,#EspertRosales2019,1,0


In [8]:
print(df0.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(273, 6)
(661, 6)
(357, 6)
(136, 6)
(215, 6)
(64, 6)


# Paso2: Eliminando menciones conjuntas

In [56]:
men_FF=['alberto','cfk','cristina','fernandez','fernández','massa','ff','alferdez'] #formas de nombrar a Alberto o a su partido
men_MM=['macri','mm','mauricio','pichetto','vidal'] #formas de nombrar a Mauricio Macri o a su partido
men_FIT=['caño','nico','nicolas','nicolás'] #formas de nombrar a Nicolas del Caño  o a su partido
men_espert=['espert','josé','luis','profesor','rosales'] #formas de nombrar a Espert o a su partido
men_lavagna=['lavagna','roberto','urtubey'] #formas de nombrar a Lavagna o a su partido
men_cent=['gomez','centurion','centurión'] #formas de nombrar a Gomez Centurión o a su partido

In [57]:
others_M= men_FF +  men_FIT + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Macri
others_FF= men_MM +  men_FIT + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Alberto Fernandez
others_LAVG= men_FF +  men_FIT + men_espert + men_MM + men_cent # mención de otros candidatos en el dataset de Lavagna
others_FIT= men_FF +  men_MM + men_espert + men_lavagna + men_cent # mención de otros candidatos en el dataset de Nicolás del Caño
others_ESP= men_FF +  men_FIT + men_MM + men_lavagna + men_cent # mención de otros candidatos en el dataset de Espert
others_CENT= men_FF +  men_FIT + men_MM + men_lavagna + men_espert # mención de otros candidatos en el dataset de Gomez Centurión

In [72]:
import re
from nltk import word_tokenize 


In [78]:
from nltk import punkt

In [82]:
def eliminador(tweet,nombres):
    temp=str(tweet)
    
    temp= re.sub(r'\W+', ' ',temp) #elimino simbolos, ya que aplicaremos esta función sobre los tweets sin procesar
                                   # mas abajo se explica el por qué de esto.
                                   
    tokens=word_tokenize(temp) #tokenizamos
    for word in tokens:
        for nombre in nombres:
            x = re.findall(nombre, word.lower()) 
            if x :
                empty=''
                return empty # si encuentra la mención un candidato ajeno al partido, se elimina el tweet
            else:
                continue # continuamos hasta encontrar una coincidencia
    return tweet #si no se encuentra una mención de otro candidato, se retorna el tweet original

In [154]:
dfa=df0.copy()
dfb=df1.copy()
dfc=df2.copy()
dfd=df3.copy()
dfe=df4.copy()
dff=df5.copy()

In [155]:
dfa.head()

,Tweets,Date,User,hashtags,Favs,RT
0,La solución es #EspertRosales2019 #YoVotoEsper...,2019-08-10 23:59:58+00:00,Ivan78229914,#EspertRosales2019 #YoVotoEspert #ChauViejaPol...,21,2
1,Final #argentina #PASO2019 predictions: @mauri...,2019-08-10 23:13:12+00:00,speevr,#argentina #PASO2019 #Espert #ArgentinaVota,30,7
2,"Nadie LO EXPUSO... cada uno, es artífice de su...",2019-08-10 23:04:01+00:00,TROOPERGRIS,,1,1
3,¡Desde España le deseamos mucha suerte al @Fre...,2019-08-10 22:46:24+00:00,libtv_,#YoVotoEspertpic,74,19
4,#EspertRosales2019 @jlespert Presidente !! @lu...,2019-08-10 22:36:02+00:00,EzequielDQuiro1,#EspertRosales2019,1,0


In [156]:
dfa['Tweets']=dfa["Tweets"].map(lambda x : eliminador(x,others_ESP))  #OJO, no aplicarlo sobre columna corregido

In [157]:
dfa.head(50)

,Tweets,Date,User,hashtags,Favs,RT
0,La solución es #EspertRosales2019 #YoVotoEsper...,2019-08-10 23:59:58+00:00,Ivan78229914,#EspertRosales2019 #YoVotoEspert #ChauViejaPol...,21,2
1,,2019-08-10 23:13:12+00:00,speevr,#argentina #PASO2019 #Espert #ArgentinaVota,30,7
2,,2019-08-10 23:04:01+00:00,TROOPERGRIS,,1,1
3,¡Desde España le deseamos mucha suerte al @Fre...,2019-08-10 22:46:24+00:00,libtv_,#YoVotoEspertpic,74,19
4,#EspertRosales2019 @jlespert Presidente !! @lu...,2019-08-10 22:36:02+00:00,EzequielDQuiro1,#EspertRosales2019,1,0
5,The winner is: @JMilei @jlespert #yovotoespert...,2019-08-10 21:55:11+00:00,Maurom09224851,#yovotoesperthttps,4,1
6,No son muchos...,2019-08-10 21:48:21+00:00,Diecale,,5,0
7,hay gente que va a votar a espert no irónicamente,2019-08-10 21:46:49+00:00,chanotfound,,358,64
8,Te dejo mi opinión y abro el debate @JMilei @l...,2019-08-10 21:32:07+00:00,bastadebiribiri,,4,1
9,Hice presencia pre-elecciones y me voy. VOTEN ...,2019-08-10 21:09:51+00:00,belumita,,324,33


In [158]:
dfa[dfa.Tweets==''].count()

Tweets      37
Date        37
User        37
hashtags    37
Favs        37
RT          37
dtype: int64

In [159]:
dfa=dfa[dfa.Tweets!='']

In [160]:
dfa.shape

(223, 6)

In [161]:
dfb['Tweets']=dfb["Tweets"].map(lambda x : eliminador(x,others_M))

In [162]:
dfb.head(25)

,Tweets,Date,User,hashtags,Favs,RT
0,Mauricio Macri's chances of being re-elected i...,2019-08-10 23:57:51+00:00,TheEconomist,,65,23
1,,2019-08-10 23:56:34+00:00,OmarAliCandanga,#SobredosisDeTV #PodemosHablar #YoTeVotoAlberto,698,397
2,Cada vez somos más los que seguimos apostando ...,2019-08-10 23:48:01+00:00,ItalianosproR,#JuntosSomosImparables #JuntosPorElCambio #Def...,13,9
3,"Decile a pa Qué pague, la deuda Dale esme",2019-08-10 23:42:05+00:00,GladysJade2,,25,2
4,"Elegimos el cambio, y todo cambio verdadero re...",2019-08-10 23:41:02+00:00,MartinGrandeok,#YoVotoMM #YoVotoMM2019 #NoVuelvenNuncaMaspic,51,31
5,A los que hablan de bolsos les digo el delincu...,2019-08-10 23:38:53+00:00,fllorenteantoni,,10544,3323
6,"Vota inteligente, vota por @mauriciomacri Los ...",2019-08-10 23:37:41+00:00,juancdula,#Argentina,151,99
7,"El domingo, ninguno de los millones de Argenti...",2019-08-10 23:36:47+00:00,sgaddaapollonio,,13883,3686
8,pic.twitter.com/vo61dXnOY6,2019-08-10 23:35:20+00:00,NAPOPSA,,11,4
9,Más allá de que te considero una persona poco ...,2019-08-10 23:33:18+00:00,danielsch06,,21,2


In [166]:
dfb=dfb[dfb.Tweets!='']
dfb.shape

(569, 6)

In [167]:
dfc['Tweets']=dfc["Tweets"].map(lambda x : eliminador(x,others_FF))

In [168]:
dfc.head(20)

,Tweets,Date,User,hashtags,Favs,RT
0,Obviously!,2019-08-10 23:42:22+00:00,Young_Lust15,,1,0
1,"Decile a la mafia d tu familia, Del DIARIO LA ...",2019-08-10 23:34:39+00:00,fabio_gonzal,,171,26
2,Volveremos,2019-08-10 23:12:57+00:00,ivyseptiembre81,,2,0
3,,2019-08-10 23:06:37+00:00,shiafatwa9,,0,1
4,Para recuperar todo lo que perdimos en estos t...,2019-08-10 23:05:20+00:00,guanitamedina,#YoTeVotoAlbertopic,184,128
5,Tu apellido es Mitre... Que podemos esperar de...,2019-08-10 23:04:30+00:00,caireydecopas,,350,29
6,IMPORTANTÍSIMO MENSAJE DE @CFKArgentina PARA T...,2019-08-10 22:57:22+00:00,rubalef,,171,145
7,"Pasa que ella es la jefa de la mafia, capo",2019-08-10 22:47:06+00:00,LaMacrista,,850,103
8,"To @CFKArgentina Hello, we have enabled a tele...",2019-08-10 22:37:27+00:00,RicardoLuisSan9,,0,0
9,"A la Argentina que viene, más justa, equitativ...",2019-08-10 22:28:32+00:00,JuanManzurOK,,317,101


In [169]:
dfc=dfc[dfc.Tweets!='']
dfc.shape

(329, 6)

In [170]:
dfd['Tweets']=dfd["Tweets"].map(lambda x : eliminador(x,others_LAVG))

In [176]:
dfd=dfd[dfd.Tweets!='']
dfd.shape

(91, 6)

In [177]:
dfe['Tweets']=dfe["Tweets"].map(lambda x : eliminador(x,others_FIT))
dfe=dfe[dfe.Tweets!='']
dfe.shape

(163, 6)

In [179]:
dff['Tweets']=dff["Tweets"].map(lambda x : eliminador(x,others_CENT))
dff=dff[dff.Tweets!='']
dff.shape

(64, 6)

# Paso 3: Creacion del corpus de entrenamiento

 Ante la necesidad de adaptar el algoritmo a las expresiones y modismos que utilizan los argentinos al momento de hablar sobre política, se decidió armar un corpus propio de entrenamiento y no hacer uso de corpus lingüísticos internacionales, como los que
provee la sociedad española de Natural Language Processing (corpus TASS).  
(De todos modos, para los interesados se deja el link para descargarlo : [corpus TASS](http://www.sepln.org/workshops/tass/tass_data/download.php?auth=SthaMsBsw4leVvKe1r9))  
Para armar este corpus político se clasificó manualmente cerca de 3 mil tweets del total de tweets scrapeados, con las leyendas positivo, negativo y ambiguo, siempre buscando que las 3 clases se encuentren balanceadas (mil tweets clasificados por
cada sentimiento).  

In [180]:
pwd

'/home/alex/Desktop/GitHub_repo/Sentiment_Analysis_Politica-Argentina'

In [181]:
ls

 datasets/                          README.md
 How_Not_to_Predict_Elections.pdf  'Scrapping Sofmanarg.ipynb'
 imagenes/                          Untitled1.ipynb
 PASO.ipynb


In [184]:
test_ds=pd.read_excel('datasets/train_dataset_FINAL.xlsx')
test_ds.head()

,Tweets,polaridad
0,"*678, DDD y Barone. *Urtubey y el premio al má...",AMBIGUO
1,*conversación con mi abuelo* -Te tengo que pas...,NEGATIVO
2,*Incomodidad,AMBIGUO
3,"""boletas nefastas"" *se levantan todos* hay uno...",NEGATIVO
4,..atreves hacer comparaciones sin matizes entr...,NEGATIVO


In [185]:
test_ds.shape

(14027, 2)

# Paso 4: Procesamiento del conjunto de entrenamiento

- Se convierte todo el texto a mínuscula
- Se eliminan todo tipo de caracteres no alfabéticos
- Se eliminan todo tipo de URL's 
- Se eliminan nombres de usuario, emojis, etc.
- Se detecta todo tipo de insulto o palabra despectiva y se lo transforma a la etiqueta 'toxicword', con el objetivo de unificar
- Se detecta la mención de algun candidato ya sea por nombre, apellido o apodo y se lo transforma a la etiqueta "somecandidate".
- Se eliminan 'stopwords' (palabras que se repiten con mucha frecuencia pero que no aportan demasiado valor sintáctico, como por ej: de, por, con, ....)
- Se unifican las risas : 'ajjajajajajaj' ----> 'jaja', 'jojojo'----> jaja
- Se lleva a cabo un spellchecking, es decir, se corrigen las palabras mal escritas debido a modismos en redes sociales.  
Ej: 'xq'--> 'porque' , 'q' ---> 'que', etc.
- Se eliminan frases que hagan referencia a cosas dichas por otras personas.  
Por ej: 'El candidato dijo que "tendremos un futuro mejor" ---> 'El candidato dijo que'


In [186]:
import pandas as pd
import numpy as np
import re